In [1]:
import pandas as pd
import numpy as np
import boto3
import psycopg2
import configparser

In [2]:
config = configparser.ConfigParser()
config.read('config.cfg')

['config.cfg']

In [3]:
RDS_HOST = 'tienda-transactional.cjykcg6gmirx.us-east-2.rds.amazonaws.com'
print(RDS_HOST)

tienda-transactional.cjykcg6gmirx.us-east-2.rds.amazonaws.com


In [4]:
postgres_driver = f"""postgresql://{config.get('TRANSACC', 'DB_USER')}:{config.get('TRANSACC', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('TRANSACC', 'DB_PORT')}/{config.get('TRANSACC', 'DB_NAME')}"""

#### Creamos Dimensión de Personas


In [6]:
sql_query = 'SELECT * FROM persona;'
dimPersonas = pd.read_sql(sql_query, postgres_driver)
dimPersonas.head()

,idpersona,tipo_persona,nombre,tipo_documento,num_documento,direccion,telefono,email
0,1104873732,cliente,Jake Shaw,Pasaporte,181319281517914,"8799 Melanie Harbor\nEast Jameston, NY 19895",31936573,clewis@gmail.com
1,1926452725,proveedor,Mark Smith,Tarjeta Residencia,148758368500703,"0138 Brett Flat Apt. 194\nLake Monicashire, NJ...",38508214,qguzman@yahoo.com
2,1481139881,proveedor,Kim Campbell,Pasaporte,106674743818680,"78414 Nelson Ridge Suite 943\nPort Justinberg,...",38550411,xwright@yahoo.com
3,1614388249,proveedor,Tammy Walker,Pasaporte,121355435980293,"0249 Jose Haven\nSouth Jonathan, NM 84268",30169562,damonturner@gmail.com
4,1890707599,proveedor,Robert Cox,NIE,170588008249640,"196 Coffey Circles\nHeatherport, MA 08627",30336897,coleashley@hotmail.com


#### Dimensión de Usuario


In [7]:
sql_query = 'SELECT * FROM usuario;'
df_usuario = pd.read_sql(sql_query, postgres_driver)
df_usuario.head()

,idusuario,idrol,nombre,tipo_documento,num_documento,direccion,telefono,email,clave,estado
0,1804123,10003,Paul Sutton,NIE,140128333983697,"111 Weber Junction Apt. 429\nBarryside, LA 94986",35441805,carrollnicholas@gmail.com,"[b'e', b'8', b'3', b'c', b'e', b'a', b'7', b'd...",0
1,8445129,10003,Kimberly Perry,Pasaporte,175442907313587,"259 Shelton Crescent\nPort Karenfurt, HI 39098",30196725,cesarandersen@hotmail.com,"[b'b', b'e', b'6', b'3', b'8', b'd', b'f', b'2...",1
2,4515979,10003,Christopher Smith,DNI,138884399972823,USNS Avila\nFPO AE 40948,39607390,montgomerycaroline@yahoo.com,"[b'9', b'8', b'1', b'b', b'e', b'0', b'c', b'e...",1
3,4599747,10003,Alyssa Shepherd,DNI,108212203657477,"083 Bush Plain\nPort Teresa, PA 71653",38033705,pmills@yahoo.com,"[b'7', b'b', b'6', b'a', b'a', b'c', b'3', b'0...",1
4,8143038,10002,Heather Mcguire,NIE,191889297037725,"29404 Monique Vista\nLake Adam, OR 71683",30832425,joneskenneth@hotmail.com,"[b'f', b'1', b'6', b'a', b'b', b'b', b'a', b'2...",0


In [8]:
sql_query = 'SELECT * FROM rol;'
df_rol = pd.read_sql(sql_query, postgres_driver)
df_rol.head()

,idrol,nombre,descripcion,estado
0,10001,Administrador,Este rol tiene acceso completo al sistema y pu...,1
1,10002,Moderador,Este rol tiene permisos para moderar contenido...,1
2,10003,Usuario Estándar,Este rol tiene acceso limitado y puede realiza...,1
3,10004,Soporte Técnico,Este rol proporciona soporte técnico y tiene a...,1
4,10005,Invitado,Este rol tiene acceso limitado y solo puede ve...,1


In [9]:
df_usuario = df_usuario.merge(df_rol, how='inner', on='idrol')
dimUsuario = df_usuario.drop(['idrol'], axis=1)
dimUsuario.head()

,idusuario,nombre_x,tipo_documento,num_documento,direccion,telefono,email,clave,estado_x,nombre_y,descripcion,estado_y
0,1804123,Paul Sutton,NIE,140128333983697,"111 Weber Junction Apt. 429\nBarryside, LA 94986",35441805,carrollnicholas@gmail.com,"[b'e', b'8', b'3', b'c', b'e', b'a', b'7', b'd...",0,Usuario Estándar,Este rol tiene acceso limitado y puede realiza...,1
1,8445129,Kimberly Perry,Pasaporte,175442907313587,"259 Shelton Crescent\nPort Karenfurt, HI 39098",30196725,cesarandersen@hotmail.com,"[b'b', b'e', b'6', b'3', b'8', b'd', b'f', b'2...",1,Usuario Estándar,Este rol tiene acceso limitado y puede realiza...,1
2,4515979,Christopher Smith,DNI,138884399972823,USNS Avila\nFPO AE 40948,39607390,montgomerycaroline@yahoo.com,"[b'9', b'8', b'1', b'b', b'e', b'0', b'c', b'e...",1,Usuario Estándar,Este rol tiene acceso limitado y puede realiza...,1
3,4599747,Alyssa Shepherd,DNI,108212203657477,"083 Bush Plain\nPort Teresa, PA 71653",38033705,pmills@yahoo.com,"[b'7', b'b', b'6', b'a', b'a', b'c', b'3', b'0...",1,Usuario Estándar,Este rol tiene acceso limitado y puede realiza...,1
4,6770927,Charles Brady,DNI,199263310165983,"979 Teresa Fords\nSouth Sydneyburgh, WI 07178",36997181,oharris@hotmail.com,"[b'e', b'3', b'd', b'c', b'f', b'0', b'8', b'a...",1,Usuario Estándar,Este rol tiene acceso limitado y puede realiza...,1


#### Dimensión Artículo


In [10]:
sql_query = 'SELECT * FROM articulo;'
df_articulo = pd.read_sql(sql_query, postgres_driver)
df_articulo.head()

,idarticulo,idcategoria,codigo,nombre,precio_venta,stock,descripcion,imagen,estado
0,0,20934,fpknjVSR,Tenis Represent,399.74,28,Artículo Tenis Represent en venta en tienda,fig_0,1
1,1,75369,iIvvrLoc,Cuadro Matter,840.24,46,Artículo Cuadro Matter en venta en tienda,fig_1,1
2,2,40879,GcRHEBYt,Lámpara Young,288.63,81,Artículo Lámpara Young en venta en tienda,fig_2,1
3,3,64218,LNbMyjQs,Maquillaje Black,830.51,60,Artículo Maquillaje Black en venta en tienda,fig_3,1
4,4,32684,eoixrXA2,Anillo Eight,796.86,15,Artículo Anillo Eight en venta en tienda,fig_4,1


In [11]:
sql_query = 'SELECT * FROM categoria;'
df_categoria = pd.read_sql(sql_query, postgres_driver)
df_categoria.head()

,idcategoria,nombre,descripcion,estado
0,39582,Ropa y Accesorios,"Incluye prendas de vestir para hombres, mujere...",1
1,82016,Calzado,"Incluye zapatos formales, tenis, botas, sandal...",1
2,51470,Juguetes y Juegos,Una amplia gama de juguetes para ninos de toda...,1
3,75369,Electronicos,"Desde telefonos moviles, computadoras portatil...",1
4,20934,Mascotas,"Ofrece productos para el cuidado de mascotas, ...",1


In [12]:
df_articulo = df_articulo.merge(df_categoria, how='inner', on='idcategoria')
dimArticulo = df_articulo.drop(['idcategoria'], axis=1)
dimArticulo.head()

,idarticulo,codigo,nombre_x,precio_venta,stock,descripcion_x,imagen,estado_x,nombre_y,descripcion_y,estado_y
0,0,fpknjVSR,Tenis Represent,399.74,28,Artículo Tenis Represent en venta en tienda,fig_0,1,Mascotas,"Ofrece productos para el cuidado de mascotas, ...",1
1,6,Z8ceLzGC,Cocina Body,758.94,95,Artículo Cocina Body en venta en tienda,fig_6,1,Mascotas,"Ofrece productos para el cuidado de mascotas, ...",1
2,18,CJnBc7Zc,Lámpara Can,481.90,84,Artículo Lámpara Can en venta en tienda,fig_18,1,Mascotas,"Ofrece productos para el cuidado de mascotas, ...",1
3,38,fvCrXCJ4,Zapatos Pass,672.75,49,Artículo Zapatos Pass en venta en tienda,fig_38,1,Mascotas,"Ofrece productos para el cuidado de mascotas, ...",1
4,63,Mnv73kg4,Bolígrafo None,296.18,28,Artículo Bolígrafo None en venta en tienda,fig_63,1,Mascotas,"Ofrece productos para el cuidado de mascotas, ...",1


#### Dimensión Venta


In [13]:
sql_query = 'SELECT * FROM venta;'
df_venta = pd.read_sql(sql_query, postgres_driver)
df_venta.head()

,idventa,idcliente,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado
0,0,1026011053,4354477,Ticket de Venta,8976937,None,2016-11-20,46.59,196.31,Cancelada
1,1,1271229612,3010227,Recibo,6737015,None,2016-11-20,1.21,85178.23,En proceso
2,2,1561688182,4073888,Nota de Crédito,3555604,None,2016-11-20,56.02,4868.43,Reembolsada
3,3,1849215225,3202497,Factura,4026162,None,2016-11-20,14.67,70004.67,En proceso
4,4,1197162828,3439940,Ticket de Venta,4729149,None,2016-11-20,95.86,7930.71,Reembolsada


In [14]:
sql_query = 'SELECT * FROM persona'
df_cliente = pd.read_sql(sql_query, postgres_driver)
df_cliente = df_cliente[df_cliente['tipo_persona']=='cliente']
df_cliente = df_cliente.rename(columns={'idpersona': 'idcliente'})
df_cliente.head()

,idcliente,tipo_persona,nombre,tipo_documento,num_documento,direccion,telefono,email
0,1104873732,cliente,Jake Shaw,Pasaporte,181319281517914,"8799 Melanie Harbor\nEast Jameston, NY 19895",31936573,clewis@gmail.com
5,1297518288,cliente,Brandy Black,Pasaporte,192179773977011,"030 Regina Keys\nJessemouth, UT 39930",30119282,matthewsimpson@gmail.com
6,1916953413,cliente,Gregory Barrett,DNI,194033334093778,"49080 William Haven Suite 555\nGilbertton, GU ...",32372975,christopher41@yahoo.com
7,1042651351,cliente,Jennifer Carpenter,DNI,119721787263718,"36499 Kayla Estate\nGregorytown, OR 39976",38485415,ronald93@gmail.com
8,1868511264,cliente,Crystal Williams,Pasaporte,187866809990432,"36743 Jennifer Passage\nLuisport, VA 87982",39993225,qroberts@hotmail.com


In [15]:
df_venta_cliente = df_venta.merge(df_cliente, how='inner', on='idcliente')
df_venta_cliente.head()

,idventa,idcliente,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado,tipo_persona,nombre,tipo_documento,num_documento,direccion,telefono,email
0,0,1026011053,4354477,Ticket de Venta,8976937,None,2016-11-20,46.59,196.31,Cancelada,cliente,Justin Macdonald,DNI,193218063994703,3521 Crawford Ridge Suite 735\nEast Michaelbur...,35252290,urusso@hotmail.com
1,3,1849215225,3202497,Factura,4026162,None,2016-11-20,14.67,70004.67,En proceso,cliente,Richard Young,Tarjeta Residencia,151675031855716,"9319 Jackson Valley Apt. 706\nFerrellborough, ...",39239929,rodneyflynn@hotmail.com
2,4,1197162828,3439940,Ticket de Venta,4729149,None,2016-11-20,95.86,7930.71,Reembolsada,cliente,James Harrison,DNI,134804193420103,"4932 Rachel Ford Suite 642\nSouth Sandrastad, ...",34199317,dawnmoyer@yahoo.com
3,339,1197162828,5501533,Boleta de Venta,3261093,None,2016-11-20,71.82,22442.35,Reembolsada,cliente,James Harrison,DNI,134804193420103,"4932 Rachel Ford Suite 642\nSouth Sandrastad, ...",34199317,dawnmoyer@yahoo.com
4,6,1549452778,2384465,Factura,9248796,None,2016-11-20,28.37,79103.99,Reembolsada,cliente,Joseph Buchanan,DNI,135943636820021,"7512 Scott Village\nPort Melissaport, DC 40363",37562725,jonathanwhitaker@yahoo.com


In [16]:
sql_query = 'SELECT * FROM usuario;'
df_usuario = pd.read_sql(sql_query, postgres_driver)
df_usuario.head()

,idusuario,idrol,nombre,tipo_documento,num_documento,direccion,telefono,email,clave,estado
0,1804123,10003,Paul Sutton,NIE,140128333983697,"111 Weber Junction Apt. 429\nBarryside, LA 94986",35441805,carrollnicholas@gmail.com,"[b'e', b'8', b'3', b'c', b'e', b'a', b'7', b'd...",0
1,8445129,10003,Kimberly Perry,Pasaporte,175442907313587,"259 Shelton Crescent\nPort Karenfurt, HI 39098",30196725,cesarandersen@hotmail.com,"[b'b', b'e', b'6', b'3', b'8', b'd', b'f', b'2...",1
2,4515979,10003,Christopher Smith,DNI,138884399972823,USNS Avila\nFPO AE 40948,39607390,montgomerycaroline@yahoo.com,"[b'9', b'8', b'1', b'b', b'e', b'0', b'c', b'e...",1
3,4599747,10003,Alyssa Shepherd,DNI,108212203657477,"083 Bush Plain\nPort Teresa, PA 71653",38033705,pmills@yahoo.com,"[b'7', b'b', b'6', b'a', b'a', b'c', b'3', b'0...",1
4,8143038,10002,Heather Mcguire,NIE,191889297037725,"29404 Monique Vista\nLake Adam, OR 71683",30832425,joneskenneth@hotmail.com,"[b'f', b'1', b'6', b'a', b'b', b'b', b'a', b'2...",0


In [17]:
df_venta_cliente_usuario = df_venta_cliente.merge(df_usuario, how='inner', on='idusuario')
dimVenta = df_venta_cliente_usuario.drop(['idcliente', 'idusuario', 'idrol'], axis=1)
dimVenta.head()

,idventa,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_x,tipo_persona,nombre_x,...,telefono_x,email_x,nombre_y,tipo_documento_y,num_documento_y,direccion_y,telefono_y,email_y,clave,estado_y
0,0,Ticket de Venta,8976937,None,2016-11-20,46.59,196.31,Cancelada,cliente,Justin Macdonald,...,35252290,urusso@hotmail.com,Christopher Brown,DNI,105963809624204,"3231 Brian Mission Apt. 533\nPort Christina, I...",31207524,pedwards@hotmail.com,"[b'5', b'c', b'5', b'1', b'4', b'a', b'c', b'd...",1
1,3,Factura,4026162,None,2016-11-20,14.67,70004.67,En proceso,cliente,Richard Young,...,39239929,rodneyflynn@hotmail.com,Keith Murillo,Pasaporte,175985114810506,"139 Stephanie Stravenue Suite 492\nSouth Jon, ...",32873870,wcox@yahoo.com,"[b'5', b'2', b'a', b'a', b'0', b'3', b'3', b'3...",0
2,4,Ticket de Venta,4729149,None,2016-11-20,95.86,7930.71,Reembolsada,cliente,James Harrison,...,34199317,dawnmoyer@yahoo.com,Zachary Diaz,Pasaporte,146968857111917,"40902 Roy Wall\nPort Nicholas, MP 62053",37082541,jclayton@gmail.com,"[b'4', b'4', b'f', b'6', b'6', b'4', b'2', b'4...",0
3,339,Boleta de Venta,3261093,None,2016-11-20,71.82,22442.35,Reembolsada,cliente,James Harrison,...,34199317,dawnmoyer@yahoo.com,Julie Werner,NIE,194658258741804,"14196 Jared Wall\nPort Jennifer, ND 87544",34316978,eford@hotmail.com,"[b'4', b'5', b'b', b'5', b'e', b'd', b'b', b'd...",1
4,337,Nota de Débito,6899451,None,2016-11-20,37.96,4710.36,Cancelada,cliente,Michael Roberts,...,39362576,stephenskaren@yahoo.com,Julie Werner,NIE,194658258741804,"14196 Jared Wall\nPort Jennifer, ND 87544",34316978,eford@hotmail.com,"[b'4', b'5', b'b', b'5', b'e', b'd', b'b', b'd...",1


#### Dimensión Ingreso


In [18]:
sql_query = 'SELECT * FROM ingreso;'
df_ingreso = pd.read_sql(sql_query, postgres_driver)
df_ingreso.head()

,idingreso,idproveedor,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado
0,0,1642280789,5459022,Boleta de Venta,166140,None,2016-11-20,87.97,97260.59,Completada
1,1,1166732562,8445129,Nota de Débito,159065,None,2016-11-20,62.16,44848.44,Pendiente
2,2,1800415359,2974339,Nota de Crédito,825756,None,2016-11-20,7.39,95382.14,Reembolsada
3,3,1269983604,3006239,Nota de Crédito,816810,None,2016-11-20,61.67,69612.12,Completada
4,4,1192371049,4601927,Boleta de Venta,119520,None,2016-11-20,72.91,7340.93,Completada


In [23]:
sql_query = 'SELECT * FROM persona;'
df_proveedor = pd.read_sql(sql_query, postgres_driver)
df_proveedor = df_proveedor[df_proveedor['tipo_persona']=='proveedor']
df_proveedor = df_proveedor.rename(columns={'idpersona': 'idproveedor'})
df_proveedor.head()

,idproveedor,tipo_persona,nombre,tipo_documento,num_documento,direccion,telefono,email
1,1926452725,proveedor,Mark Smith,Tarjeta Residencia,148758368500703,"0138 Brett Flat Apt. 194\nLake Monicashire, NJ...",38508214,qguzman@yahoo.com
2,1481139881,proveedor,Kim Campbell,Pasaporte,106674743818680,"78414 Nelson Ridge Suite 943\nPort Justinberg,...",38550411,xwright@yahoo.com
3,1614388249,proveedor,Tammy Walker,Pasaporte,121355435980293,"0249 Jose Haven\nSouth Jonathan, NM 84268",30169562,damonturner@gmail.com
4,1890707599,proveedor,Robert Cox,NIE,170588008249640,"196 Coffey Circles\nHeatherport, MA 08627",30336897,coleashley@hotmail.com
10,1457625610,proveedor,Hailey Schultz,NIE,117173669173401,"1188 Troy Village Suite 416\nBelindaborough, R...",32493693,michaelbennett@yahoo.com


In [24]:
df_ingreso_proveedor = df_ingreso.merge(df_proveedor, how='inner', on='idproveedor')
df_ingreso_proveedor.head()

,idingreso,idproveedor,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado,tipo_persona,nombre,tipo_documento,num_documento,direccion,telefono,email
0,2,1800415359,2974339,Nota de Crédito,825756,None,2016-11-20,7.39,95382.14,Reembolsada,proveedor,Richard Hayes,DNI,199824286702586,"0180 Amy Mountains Suite 724\nLake Kellyberg, ...",33841403,walter67@yahoo.com
1,796,1800415359,6885038,Factura,213921,None,2016-11-20,6.99,33995.06,En proceso,proveedor,Richard Hayes,DNI,199824286702586,"0180 Amy Mountains Suite 724\nLake Kellyberg, ...",33841403,walter67@yahoo.com
2,901,1800415359,5415735,Factura,392871,None,2016-11-20,30.33,8309.38,Cancelada,proveedor,Richard Hayes,DNI,199824286702586,"0180 Amy Mountains Suite 724\nLake Kellyberg, ...",33841403,walter67@yahoo.com
3,3,1269983604,3006239,Nota de Crédito,816810,None,2016-11-20,61.67,69612.12,Completada,proveedor,Tracy Hernandez,DNI,172992765957082,"7227 Daniel Gardens\nWest Christopherland, IL ...",35383803,asmith@hotmail.com
4,9,1031016474,1475899,Nota de Débito,508290,None,2016-11-20,18.90,26343.35,En proceso,proveedor,Julie Porter,Tarjeta Residencia,155368546911216,Unit 7699 Box 8921\nDPO AA 33819,36577918,millerkendra@yahoo.com


In [25]:
sql_query = 'SELECT * FROM usuario'
df_usuario = pd.read_sql(sql_query, postgres_driver)
df_usuario.head()

,idusuario,idrol,nombre,tipo_documento,num_documento,direccion,telefono,email,clave,estado
0,1804123,10003,Paul Sutton,NIE,140128333983697,"111 Weber Junction Apt. 429\nBarryside, LA 94986",35441805,carrollnicholas@gmail.com,"[b'e', b'8', b'3', b'c', b'e', b'a', b'7', b'd...",0
1,8445129,10003,Kimberly Perry,Pasaporte,175442907313587,"259 Shelton Crescent\nPort Karenfurt, HI 39098",30196725,cesarandersen@hotmail.com,"[b'b', b'e', b'6', b'3', b'8', b'd', b'f', b'2...",1
2,4515979,10003,Christopher Smith,DNI,138884399972823,USNS Avila\nFPO AE 40948,39607390,montgomerycaroline@yahoo.com,"[b'9', b'8', b'1', b'b', b'e', b'0', b'c', b'e...",1
3,4599747,10003,Alyssa Shepherd,DNI,108212203657477,"083 Bush Plain\nPort Teresa, PA 71653",38033705,pmills@yahoo.com,"[b'7', b'b', b'6', b'a', b'a', b'c', b'3', b'0...",1
4,8143038,10002,Heather Mcguire,NIE,191889297037725,"29404 Monique Vista\nLake Adam, OR 71683",30832425,joneskenneth@hotmail.com,"[b'f', b'1', b'6', b'a', b'b', b'b', b'a', b'2...",0


In [26]:
dimIngreso = df_ingreso_proveedor.merge(df_usuario, how='inner', on='idusuario')
dimIngreso.drop(['idproveedor', 'idusuario', 'idrol'], axis=1,  inplace=True)
dimIngreso.head()

,idingreso,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_x,tipo_persona,nombre_x,...,telefono_x,email_x,nombre_y,tipo_documento_y,num_documento_y,direccion_y,telefono_y,email_y,clave,estado_y
0,2,Nota de Crédito,825756,None,2016-11-20,7.39,95382.14,Reembolsada,proveedor,Richard Hayes,...,33841403,walter67@yahoo.com,Brian Graves,DNI,115972117432456,"903 Reginald Parkways\nHughesburgh, AK 94569",39393879,john68@yahoo.com,"[b'e', b'b', b'd', b'2', b'7', b'9', b'0', b'a...",1
1,796,Factura,213921,None,2016-11-20,6.99,33995.06,En proceso,proveedor,Richard Hayes,...,33841403,walter67@yahoo.com,Jonathan Johnson,NIE,143029828139019,569 Cameron Overpass Suite 224\nNew Jacqueline...,31283830,joycediaz@hotmail.com,"[b'0', b'a', b'7', b'a', b'0', b'7', b'4', b'8...",1
2,901,Factura,392871,None,2016-11-20,30.33,8309.38,Cancelada,proveedor,Richard Hayes,...,33841403,walter67@yahoo.com,Keith Richards,NIE,147842072428762,"187 James Views Apt. 975\nSouth Donald, MN 47871",35416947,cynthia60@hotmail.com,"[b'5', b'b', b'4', b'9', b'2', b'6', b'8', b'1...",0
3,3,Nota de Crédito,816810,None,2016-11-20,61.67,69612.12,Completada,proveedor,Tracy Hernandez,...,35383803,asmith@hotmail.com,Karen Bowers,Pasaporte,137198667116663,"684 Herrera Rapids Apt. 536\nGreenberg, PA 42710",39668488,matthewmiller@hotmail.com,"[b'6', b'0', b'3', b'a', b'1', b'4', b'7', b'2...",1
4,9,Nota de Débito,508290,None,2016-11-20,18.90,26343.35,En proceso,proveedor,Julie Porter,...,36577918,millerkendra@yahoo.com,Steven Evans,DNI,199482351052642,"60351 Bullock Ports Apt. 725\nCardenasmouth, M...",36792694,michaelrobbins@yahoo.com,"[b'4', b'1', b'3', b'a', b'2', b'0', b'5', b'0...",0


#### Dimensión Detalle Venta

In [29]:
sql_query = 'SELECT * FROM detalle_venta;'
df_detalle_venta = pd.read_sql(sql_query, postgres_driver)
df_detalle_venta.head()

,iddetalle_venta,idventa,idarticulo,cantidad,precio,descuento
0,0,355,19,15,701.41,864.47
1,1,151,593,2,426.32,226.45
2,2,853,167,0,518.02,1.47
3,3,27,589,1,209.22,335.56
4,4,336,247,36,803.95,707.20


In [30]:
sql_query = 'SELECT * FROM venta;'
df_venta = pd.read_sql(sql_query, postgres_driver)
df_venta.head()

,idventa,idcliente,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado
0,0,1026011053,4354477,Ticket de Venta,8976937,None,2016-11-20,46.59,196.31,Cancelada
1,1,1271229612,3010227,Recibo,6737015,None,2016-11-20,1.21,85178.23,En proceso
2,2,1561688182,4073888,Nota de Crédito,3555604,None,2016-11-20,56.02,4868.43,Reembolsada
3,3,1849215225,3202497,Factura,4026162,None,2016-11-20,14.67,70004.67,En proceso
4,4,1197162828,3439940,Ticket de Venta,4729149,None,2016-11-20,95.86,7930.71,Reembolsada


In [31]:
df_detalle_venta_venta = df_detalle_venta.merge(df_venta, how='inner', on='idventa')
df_detalle_venta_venta.head()

,iddetalle_venta,idventa,idarticulo,cantidad,precio,descuento,idcliente,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado
0,0,355,19,15,701.41,864.47,1552735865,5873517,Boleta de Venta,3283178,None,2016-11-20,22.54,67654.09,En proceso
1,484,355,497,88,907.74,790.71,1552735865,5873517,Boleta de Venta,3283178,None,2016-11-20,22.54,67654.09,En proceso
2,2287,355,810,48,120.01,718.31,1552735865,5873517,Boleta de Venta,3283178,None,2016-11-20,22.54,67654.09,En proceso
3,2291,355,13,3,840.70,736.62,1552735865,5873517,Boleta de Venta,3283178,None,2016-11-20,22.54,67654.09,En proceso
4,4554,355,665,93,919.25,789.85,1552735865,5873517,Boleta de Venta,3283178,None,2016-11-20,22.54,67654.09,En proceso


In [32]:
sql_query = 'SELECT *FROM articulo;'
df_articulo = pd.read_sql(sql_query, postgres_driver)
df_articulo.head()

,idarticulo,idcategoria,codigo,nombre,precio_venta,stock,descripcion,imagen,estado
0,0,20934,fpknjVSR,Tenis Represent,399.74,28,Artículo Tenis Represent en venta en tienda,fig_0,1
1,1,75369,iIvvrLoc,Cuadro Matter,840.24,46,Artículo Cuadro Matter en venta en tienda,fig_1,1
2,2,40879,GcRHEBYt,Lámpara Young,288.63,81,Artículo Lámpara Young en venta en tienda,fig_2,1
3,3,64218,LNbMyjQs,Maquillaje Black,830.51,60,Artículo Maquillaje Black en venta en tienda,fig_3,1
4,4,32684,eoixrXA2,Anillo Eight,796.86,15,Artículo Anillo Eight en venta en tienda,fig_4,1


In [34]:
dimDetalleVenta = df_detalle_venta_venta.merge(df_articulo, how='inner', on='idarticulo')
dimDetalleVenta.drop(['idventa', 'idarticulo', 'idcliente', 'idusuario'], axis=1,  inplace=True)
dimDetalleVenta.head()

,iddetalle_venta,cantidad,precio,descuento,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_x,idcategoria,codigo,nombre,precio_venta,stock,descripcion,imagen,estado_y
0,0,15,701.41,864.47,Boleta de Venta,3283178,None,2016-11-20,22.54,67654.09,En proceso,97521,VTnmYNqn,Cuadro Painting,701.41,71,Artículo Cuadro Painting en venta en tienda,fig_19,1
1,192,98,701.41,969.04,Nota de Débito,7135847,None,2016-11-20,3.59,81539.99,Reembolsada,97521,VTnmYNqn,Cuadro Painting,701.41,71,Artículo Cuadro Painting en venta en tienda,fig_19,1
2,2442,77,701.41,69.62,Recibo,9069523,None,2016-11-20,94.45,25031.04,Reembolsada,97521,VTnmYNqn,Cuadro Painting,701.41,71,Artículo Cuadro Painting en venta en tienda,fig_19,1
3,2578,62,701.41,576.59,Ticket de Venta,6097629,None,2016-11-20,64.29,31191.04,En proceso,97521,VTnmYNqn,Cuadro Painting,701.41,71,Artículo Cuadro Painting en venta en tienda,fig_19,1
4,884,68,701.41,344.87,Nota de Débito,9315663,None,2016-11-20,15.54,26491.56,En proceso,97521,VTnmYNqn,Cuadro Painting,701.41,71,Artículo Cuadro Painting en venta en tienda,fig_19,1


#### Dimensión Detalle Ingreso


In [35]:
sql_query = 'SELECT * FROM detalle_ingreso;'
df_detalle_ingreso = pd.read_sql(sql_query, postgres_driver)
df_detalle_ingreso.head()

,iddetalle_ingreso,idingreso,idarticulo,cantidad,precio
0,0,56,74,66,799.37
1,1,187,509,53,546.70
2,2,138,81,65,889.49
3,3,643,290,97,242.54
4,4,393,283,50,369.07


In [36]:
sql_query = 'SELECT * FROM ingreso;'
df_ingreso = pd.read_sql(sql_query, postgres_driver)
df_ingreso.head()

,idingreso,idproveedor,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado
0,0,1642280789,5459022,Boleta de Venta,166140,None,2016-11-20,87.97,97260.59,Completada
1,1,1166732562,8445129,Nota de Débito,159065,None,2016-11-20,62.16,44848.44,Pendiente
2,2,1800415359,2974339,Nota de Crédito,825756,None,2016-11-20,7.39,95382.14,Reembolsada
3,3,1269983604,3006239,Nota de Crédito,816810,None,2016-11-20,61.67,69612.12,Completada
4,4,1192371049,4601927,Boleta de Venta,119520,None,2016-11-20,72.91,7340.93,Completada


In [38]:
df_detalle_ingreso_ingreso = df_detalle_ingreso.merge(df_ingreso, how='inner', on='idingreso')
df_detalle_ingreso_ingreso.head()

,iddetalle_ingreso,idingreso,idarticulo,cantidad,precio,idproveedor,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado
0,0,56,74,66,799.37,1200747923,5356423,Factura,564175,None,2016-11-20,55.94,95972.19,Cancelada
1,1,187,509,53,546.70,1284837395,3991744,Nota de Débito,590529,None,2016-11-20,77.11,20049.26,Cancelada
2,2,138,81,65,889.49,1487422283,4785864,Factura,476246,None,2016-11-20,82.43,97670.46,Reembolsada
3,992,138,426,8,571.12,1487422283,4785864,Factura,476246,None,2016-11-20,82.43,97670.46,Reembolsada
4,3,643,290,97,242.54,1592497923,1060392,Factura,689296,None,2016-11-20,37.38,96137.05,Reembolsada


In [39]:
sql_query = 'SELECT * FROM articulo;'
df_articulo = pd.read_sql(sql_query, postgres_driver)
df_articulo.head()

,idarticulo,idcategoria,codigo,nombre,precio_venta,stock,descripcion,imagen,estado
0,0,20934,fpknjVSR,Tenis Represent,399.74,28,Artículo Tenis Represent en venta en tienda,fig_0,1
1,1,75369,iIvvrLoc,Cuadro Matter,840.24,46,Artículo Cuadro Matter en venta en tienda,fig_1,1
2,2,40879,GcRHEBYt,Lámpara Young,288.63,81,Artículo Lámpara Young en venta en tienda,fig_2,1
3,3,64218,LNbMyjQs,Maquillaje Black,830.51,60,Artículo Maquillaje Black en venta en tienda,fig_3,1
4,4,32684,eoixrXA2,Anillo Eight,796.86,15,Artículo Anillo Eight en venta en tienda,fig_4,1


In [40]:
dimDetalleIngreso = df_detalle_ingreso_ingreso.merge(df_articulo, how='inner', on='idarticulo')
dimDetalleIngreso.drop(['idingreso', 'idarticulo', 'idproveedor', 'idusuario'], axis=1,  inplace=True)
dimDetalleIngreso.head()

,iddetalle_ingreso,cantidad,precio,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_x,idcategoria,codigo,nombre,precio_venta,stock,descripcion,imagen,estado_y
0,0,66,799.37,Factura,564175,None,2016-11-20,55.94,95972.19,Cancelada,40879,zndOOOd3,Zapatos Cause,799.37,99,Artículo Zapatos Cause en venta en tienda,fig_74,1
1,507,6,799.37,Boleta de Venta,166140,None,2016-11-20,87.97,97260.59,Completada,40879,zndOOOd3,Zapatos Cause,799.37,99,Artículo Zapatos Cause en venta en tienda,fig_74,1
2,523,40,799.37,Nota de Crédito,155143,None,2016-11-20,95.31,31893.97,Pendiente,40879,zndOOOd3,Zapatos Cause,799.37,99,Artículo Zapatos Cause en venta en tienda,fig_74,1
3,1,53,546.70,Nota de Débito,590529,None,2016-11-20,77.11,20049.26,Cancelada,75369,N4QOX3S5,Cuadro Task,546.70,4,Artículo Cuadro Task en venta en tienda,fig_509,1
4,584,81,546.70,Nota de Débito,339527,None,2016-11-20,14.76,73018.77,Cancelada,75369,N4QOX3S5,Cuadro Task,546.70,4,Artículo Cuadro Task en venta en tienda,fig_509,1


#### Dimensión Rol


In [41]:
sql_query = 'SELECT * FROM rol;'
dimRol = pd.read_sql(sql_query, postgres_driver)
dimRol.head()

,idrol,nombre,descripcion,estado
0,10001,Administrador,Este rol tiene acceso completo al sistema y pu...,1
1,10002,Moderador,Este rol tiene permisos para moderar contenido...,1
2,10003,Usuario Estándar,Este rol tiene acceso limitado y puede realiza...,1
3,10004,Soporte Técnico,Este rol proporciona soporte técnico y tiene a...,1
4,10005,Invitado,Este rol tiene acceso limitado y solo puede ve...,1


#### Dimensión Categoría


In [42]:
sql_query = 'SELECT * FROM categoria;'
dimCategoria = pd.read_sql(sql_query, postgres_driver)
dimCategoria.head()

,idcategoria,nombre,descripcion,estado
0,39582,Ropa y Accesorios,"Incluye prendas de vestir para hombres, mujere...",1
1,82016,Calzado,"Incluye zapatos formales, tenis, botas, sandal...",1
2,51470,Juguetes y Juegos,Una amplia gama de juguetes para ninos de toda...,1
3,75369,Electronicos,"Desde telefonos moviles, computadoras portatil...",1
4,20934,Mascotas,"Ofrece productos para el cuidado de mascotas, ...",1
